In [1]:
import pickle
import sys
sys.path.append("../")
from Join_scheme.data_prepare import process_stats_data
from BayesCard.Models.Bayescard_BN import Bayescard_BN
import time
import pandas as pd
import numpy as np
from BayesCard.Evaluation.cardinality_estimation import parse_query_single_table

In [3]:
from Join_scheme.data_prepare import process_stats_data
data_path = "/home/ubuntu/End-to-End-CardEst-Benchmark/datasets/stats_simplified/{}.csv"
model_folder = "/home/ubuntu/data_CE/saved_models"
data, null_values, key_attrs, table_buckets, equivalent_keys, schema, bin_size = process_stats_data(data_path,
                                                model_folder, 200, "sub_optimal", return_bin_means=False)

bucketizing equivalent key group: {'postLinks.RelatedPostId', 'posts.Id', 'tags.ExcerptPostId', 'postHistory.PostId', 'postLinks.PostId', 'votes.PostId', 'comments.PostId'}
bucketizing equivalent key group: {'comments.UserId', 'postHistory.UserId', 'posts.OwnerUserId', 'badges.UserId', 'users.Id', 'votes.UserId'}


In [8]:
def learn_histogram(data, key_attrs, bin_size=100):
    all_histogram = dict()
    all_boundary = dict()
    for table in data:
        all_histogram[table] = dict()
        all_boundary[table] = dict()
        for attr in data[table]:
            if attr in key_attrs[table]:
                continue
                #assert all_bin_size[table][attr] == len(all_bin_means[attr]) == len(all_bin_width[attr])
            else:
                hist, curr_bins = np.histogram(data[table][attr].values, bins=bin_size)
                all_histogram[table][attr] = hist/np.sum(hist)
                all_boundary[table][attr] = curr_bins
    return all_histogram, all_boundary

In [10]:
all_histogram, all_boundary = learn_histogram(data, key_attrs, bin_size=50)

In [30]:
import numpy as np
import copy

from Join_scheme.join_graph import process_condition, get_join_hyper_graph
from Join_scheme.data_prepare import identify_key_values
from BayesCard.Evaluation.cardinality_estimation import timestamp_transorform, construct_table_query


OPS = {
    '>': np.greater,
    '<': np.less,
    '>=': np.greater_equal,
    '<=': np.less_equal,
    '=': np.equal,
    '==': np.equal
}

class Factor:
    """
    This the class defines a multidimensional conditional probability.
    """
    def __init__(self, variables, pdfs, equivalent_variables=[]):
        self.variables = variables
        self.equivalent_variables = equivalent_variables
        self.pdfs = pdfs
        self.cardinalities = dict()
        for i, var in enumerate(self.variables):
            self.cardinalities[var] = pdfs.shape[i]
            if len(equivalent_variables) != 0:
                self.cardinalities[equivalent_variables[i]] = pdfs.shape[i]


class Bound_ensemble:
    """
    This the class where we store all the trained models and perform inference on the bound.
    """
    def __init__(self, bns, hist, boundary, all_bin_means, schema):
        self.bns = bns
        self.hist = hist
        self.boundary = boundary
        self.table_buckets = all_bin_means
        self.schema = schema
        self.all_keys, self.equivalent_keys = identify_key_values(schema)

    def parse_query_simple(self, query):
        """
        If your selection query contains no aggregation and nested sub-queries, you can use this function to parse a
        join query. Otherwise, use parse_query function.
        """
        query = query.replace(" where ", " WHERE ")
        query = query.replace(" from ", " FROM ")
        query = query.replace(" and ", " AND ")
        query = query.split(";")[0]
        query = query.strip()
        tables_all = {}
        join_cond = []
        table_probs = {}
        join_keys = {}
        tables_str = query.split(" WHERE ")[0].split(" FROM ")[-1]
        for table_str in tables_str.split(","):
            table_str = table_str.strip()
            if " as " in table_str:
                tables_all[table_str.split(" as ")[-1]] = table_str.split(" as ")[0]
            else:
                tables_all[table_str.split(" ")[-1]] = table_str.split(" ")[0]

        # processing conditions
        conditions = query.split(" WHERE ")[-1].split(" AND ")
        for cond in conditions:
            table, cond, join, join_key = process_condition(cond, tables_all)
            if table not in table_probs:
                table_probs[table] = 1
            if not join:
                attr = cond[0]
                op = cond[1]
                value = cond[2]
                if "Date" in attr:
                    assert "::timestamp" in value
                    value = timestamp_transorform(value.strip().split("::timestamp")[0])
                curr_prob = 0
                for i in range(0, len(self.boundary[table][attr])-1):
                    if OPS[op](self.boundary[table][attr][i], value):
                        curr_prob += self.hist[table][attr][i]
                table_probs[table] *= curr_prob
                #construct_table_query(self.bns[table], table_query[table], attr, op, value)
            else:
                join_cond.append(cond)
                for tab in join_key:
                    if tab in join_keys:
                        join_keys[tab].add(join_key[tab])
                    else:
                        join_keys[tab] = set([join_key[tab]])
        final_probs = 1
        for table in table_probs:
            final_probs *= table_probs[table]
        #print(final_probs)
        if final_probs == 0:
            final_probs = 0.001
        return tables_all, final_probs, join_cond, join_keys

    def get_all_id_conidtional_distribution(self, table_queries, join_keys, equivalent_group):
        res = dict()
        for table in join_keys:
            key_attrs = list(join_keys[table])
            if table in table_queries:
                table_query = table_queries[table]
            else:
                table_query = {}
            id_attrs, probs = self.bns[table].query_id_prob(table_query, key_attrs)
            new_id_attrs = []
            for K in id_attrs:
                for PK in equivalent_group:
                    if K in equivalent_group[PK]:
                        new_id_attrs.append(PK)
            assert len(new_id_attrs) == len(id_attrs)
            print(np.sum(probs))
            res[table] = Factor(id_attrs, probs, new_id_attrs)
        return res

    def eliminate_one_key_group(self, tables, key_group, factors, relevant_keys):
        """This version only supports 2D distributions"""
        rest_group = None
        rest_group_cardinalty = 0
        eliminated_tables = []
        rest_group_tables = []
        for table in tables:
            assert key_group in factors[table].equivalent_variables
            temp = copy.deepcopy(factors[table].equivalent_variables)
            temp.remove(key_group)
            if len(temp) == 0:
                eliminated_tables.append(table)
            for key in temp:
                if rest_group:
                    assert factors[table].cardinalities[key] == rest_group_cardinalty
                    rest_group_tables.append(table)
                else:
                    rest_group = key
                    rest_group_cardinalty = factors[table].cardinalities[key]
                    rest_group_tables = [table]

        all_probs_eliminated = []
        all_modes_eliminated = []
        for table in eliminated_tables:
            bin_modes = self.table_buckets[table].oned_bin_modes[relevant_keys[key_group][table]]
            all_probs_eliminated.append(factors[table].pdfs)
            all_modes_eliminated.append(np.minimum(bin_modes, factors[table].pdfs))
        if rest_group:
            new_factor_pdf = np.zeros(rest_group_cardinalty)
        else:
            return self.compute_bound_oned(all_probs_eliminated, all_modes_eliminated)

        for i in range(rest_group_cardinalty):
            rest_group_probs_eliminated = []
            rest_group_modes_eliminated = []
            for table in rest_group_tables:

                idx_f = factors[table].equivalent_variables.index(key_group)
                idx_b = self.table_buckets[table].id_attributes.index(relevant_keys[key_group][table])
                bin_modes = self.table_buckets[table].twod_bin_modes[relevant_keys[key_group][table]]
                if idx_f == 0 and idx_b == 0:
                    rest_group_probs_eliminated.append(factors[table].pdfs[:, i])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[:, i], factors[table].pdfs[:, i]))
                elif idx_f == 0 and idx_b == 1:
                    rest_group_probs_eliminated.append(factors[table].pdfs[:, i])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[i, :], factors[table].pdfs[:, i]))
                elif idx_f == 1 and idx_b == 0:
                    rest_group_probs_eliminated.append(factors[table].pdfs[i, :])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[:, i], factors[table].pdfs[i, :]))
                else:
                    rest_group_probs_eliminated.append(factors[table].pdfs[i, :])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[i, :], factors[table].pdfs[i, :]))
            new_factor_pdf[i] = self.compute_bound_oned(all_probs_eliminated + rest_group_probs_eliminated,
                                                        all_modes_eliminated + rest_group_modes_eliminated)

        for table in rest_group_tables:
            factors[table] = Factor([rest_group], new_factor_pdf, [rest_group])

        return None

    def compute_bound_oned(self, all_probs, all_modes):
        all_probs = np.stack(all_probs, axis=0)
        all_modes = np.stack(all_modes, axis=0)
        multiplier = np.prod(all_modes, axis=0)
        non_zero_idx = np.where(multiplier != 0)[0]
        min_number = np.amin(all_probs[:, non_zero_idx]/all_modes[:, non_zero_idx], axis=0)
        multiplier[non_zero_idx] = multiplier[non_zero_idx] * min_number
        return np.sum(multiplier)

    def get_optimal_elimination_order(self, equivalent_group, join_keys, factors):
        cardinalities = dict()
        lengths = dict()
        tables_involved = dict()
        relevant_keys = dict()
        for group in equivalent_group:
            relevant_keys[group] = dict()
            lengths[group] = len(equivalent_group[group])
            cardinalities[group] = []
            tables_involved[group] = set([])
            for keys in equivalent_group[group]:
                for table in join_keys:
                    if keys in join_keys[table]:
                        cardinalities[group].append(len(join_keys[table]))
                        tables_involved[group].add(table)
                        variables = factors[table].variables
                        variables[variables.index(keys)] = group
                        factors[table].variables = variables
                        relevant_keys[group][table] = keys
                        break
            cardinalities[group] = np.asarray(cardinalities[group])

        optimal_order = list(equivalent_group.keys())
        for i in range(len(optimal_order)):
            min_idx = i
            for j in range(i+1, len(optimal_order)):
                min_group = optimal_order[min_idx]
                curr_group = optimal_order[j]
                if np.max(cardinalities[curr_group]) < np.max(cardinalities[min_group]):
                    min_idx = j
                else:
                    min_max_tables = np.max(cardinalities[min_group])
                    min_num_max_tables = len(np.where(cardinalities[min_group] == min_max_tables)[0])
                    curr_max_tables = np.max(cardinalities[curr_group])
                    curr_num_max_tables = len(np.where(cardinalities[curr_group] == curr_max_tables)[0])
                    if curr_num_max_tables < min_num_max_tables:
                        min_idx = j
                    elif lengths[curr_group] < lengths[min_group]:
                        min_idx = j
            optimal_order[i], optimal_order[min_idx] = optimal_order[min_idx], optimal_order[i]
        return optimal_order, tables_involved, relevant_keys

    def get_cardinality_bound(self, query_str):
        tables_all, table_probs, join_cond, join_keys = self.parse_query_simple(query_str)
        equivalent_group = get_join_hyper_graph(join_keys, self.equivalent_keys)
        conditional_factors = self.get_all_id_conidtional_distribution({}, join_keys, equivalent_group)
        optimal_order, tables_involved, relevant_keys = self.get_optimal_elimination_order(equivalent_group, join_keys,
                                                                            conditional_factors)

        for key_group in optimal_order:
            tables = tables_involved[key_group]
            res = self.eliminate_one_key_group(tables, key_group, conditional_factors, relevant_keys)
        return res * table_probs


In [31]:
model_path = "/home/ubuntu/data_CE/CE_scheme_models/model_stats_sub_optimal_200.pkl"
with open(model_path, "rb") as f:
    old_BE = pickle.load(f)

In [32]:
BE = Bound_ensemble(old_BE.bns, all_histogram, all_boundary, table_buckets, schema)

In [33]:
query_file = "/home/ubuntu/End-to-End-CardEst-Benchmark/workloads/stats_CEB/sub_plan_queries/stats_CEB_sub_queries.sql"
with open(query_file, "r") as f:
    queries = f.readlines()

In [35]:
qerror = []
latency = []
pred = []
for i, query_str in enumerate(queries):
    #if i == 10: break
    query = query_str.split("||")[0][:-1]
    print("========================")
    true_card = int(query_str.split("||")[-1])
    t = time.time()
    res = BE.get_cardinality_bound(query)
    pred.append(res)
    latency.append(time.time() - t)
    qerror.append(res/true_card)
    print(f"estimating query {i}: predicted {res}, true_card {true_card}, qerror {res/true_card}, latency {time.time() - t}")

79851.0
28977.0
estimating query 0: predicted 79850.99999999997, true_card 79851, qerror 0.9999999999999997, latency 0.0034134387969970703
171470.0
28977.0
79851.0
estimating query 1: predicted 16473872.009064566, true_card 10220614, qerror 1.61182801826432, latency 0.0066471099853515625
277348.0
28977.0
171470.0
estimating query 2: predicted 34237494.045088634, true_card 1458075, qerror 23.481298318048545, latency 0.006157875061035156
277348.0
28977.0
171470.0
estimating query 3: predicted 36365326.16924744, true_card 1709781, qerror 21.268996537712983, latency 0.007519245147705078
171470.0
28977.0
34773.0
estimating query 4: predicted 12720775.199449241, true_card 7491903, qerror 1.6979364521202744, latency 0.006625652313232422
79851.0
28977.0
90373.0
estimating query 5: predicted 75500.79376166269, true_card 428612, qerror 0.17615184306940238, latency 0.006047487258911133
171470.0
28977.0
90373.0
estimating query 6: predicted 58735181.79221389, true_card 55900138, qerror 1.050716221

11102.0
91976.0
328064.0
174305.0
estimating query 82: predicted 1849.075, true_card 176191, qerror 0.01049471879948465, latency 0.003861665725708008
91976.0
174305.0
estimating query 83: predicted 172134.99999999994, true_card 170249, qerror 1.0110778917937842, latency 0.0015895366668701172
91976.0
11102.0
174305.0
estimating query 84: predicted 282288.36433263525, true_card 32792, qerror 8.608452193603172, latency 0.0028526782989501953
91976.0
11102.0
estimating query 85: predicted 11102.0, true_card 11102, qerror 1.0, latency 0.0019168853759765625
90373.0
28977.0
estimating query 86: predicted 88212.56424054553, true_card 77059, qerror 1.1447405785248386, latency 0.0057353973388671875
91976.0
11102.0
174305.0
estimating query 87: predicted 282288.36433263525, true_card 32792, qerror 8.608452193603172, latency 0.002843618392944336
90373.0
28977.0
174305.0
estimating query 88: predicted 167757.4238102492, true_card 142137, qerror 1.1802516150632782, latency 0.009370565414428711
90373.

34773.0
estimating query 143: predicted 12686425.072757138, true_card 12470361, qerror 1.0173262083396895, latency 0.008357524871826172
277348.0
28977.0
estimating query 144: predicted 252047.55944898995, true_card 140071, qerror 1.7994271437270382, latency 0.004292011260986328
34773.0
28977.0
estimating query 145: predicted 33883.84228692003, true_card 20007, qerror 1.6935993545719012, latency 0.005430459976196289
277348.0
28977.0
171470.0
34773.0
estimating query 146: predicted 52265840348.86465, true_card 52988984588, qerror 0.9863529326942582, latency 0.010511159896850586
277348.0
28977.0
171470.0
estimating query 147: predicted 236618149.21881646, true_card 2038871, qerror 116.05351648967319, latency 0.0066683292388916016
171470.0
28977.0
34773.0
estimating query 148: predicted 13344796.342335295, true_card 221701, qerror 60.19276567239343, latency 0.006906270980834961
277348.0
28977.0
34773.0
estimating query 149: predicted 12604150.290430678, true_card 425598, qerror 29.61515394

171470.0
79851.0
estimating query 212: predicted 10003879006.160835, true_card 105742741, qerror 94.60582269340678, latency 0.006952524185180664
28977.0
277348.0
171470.0
estimating query 213: predicted 32916007.96376652, true_card 1423047, qerror 23.130654127211905, latency 0.0071408748626708984
28977.0
171470.0
79851.0
estimating query 214: predicted 16090910.364930833, true_card 9976404, qerror 1.6128968278480735, latency 0.004999399185180664
28977.0
277348.0
79851.0
estimating query 215: predicted 2288762.7995142625, true_card 443440, qerror 5.1613810200123185, latency 0.0033524036407470703
28977.0
277348.0
171470.0
79851.0
estimating query 216: predicted 10003879006.160835, true_card 105742741, qerror 94.60582269340678, latency 0.00779414176940918
28977.0
277348.0
171470.0
estimating query 217: predicted 263652.113, true_card 69114343, qerror 0.0038147235661344565, latency 0.005835294723510742
28977.0
171470.0
79851.0
estimating query 218: predicted 16457710.49841628, true_card 10

34773.0
estimating query 280: predicted 14140.993999999999, true_card 1291, qerror 10.95351975213013, latency 0.006251811981201172
28977.0
277348.0
34773.0
79851.0
estimating query 281: predicted 3218904.5849999995, true_card 503641171, qerror 0.0063912657867281454, latency 0.00547480583190918
28977.0
277348.0
79851.0
estimating query 282: predicted 16425.601, true_card 6576, qerror 2.4978103710462287, latency 0.003940582275390625
28977.0
34773.0
79851.0
estimating query 283: predicted 1596.558, true_card 2275, qerror 0.7017837362637362, latency 0.007378101348876953
28977.0
277348.0
34773.0
79851.0
estimating query 284: predicted 3218904.5849999995, true_card 6272, qerror 513.2182055165815, latency 0.006956577301025391
91976.0
174305.0
estimating query 285: predicted 1520.8517414165563, true_card 25695, qerror 0.059188625857814994, latency 0.0021576881408691406
91976.0
11102.0
174305.0
estimating query 286: predicted 294259.2099968985, true_card 31580, qerror 9.317897719977786, latency

...

estimating query 2353: predicted 255572.0395062262, true_card 1827833, qerror 0.13982242333201458, latency 0.011791706085205078
171470.0
28977.0
91976.0
11102.0
estimating query 2354: predicted 246638.53374075994, true_card 19469, qerror 12.668269235233446, latency 0.008946657180786133
91976.0
303187.00000000006
174305.0
328064.0
estimating query 2355: predicted 17257764.836291317, true_card 2763952, qerror 6.2438728444963285, latency 0.00447392463684082
91976.0
303187.00000000006
171470.0
28977.0
79851.0
estimating query 2356: predicted 18047923.98316472, true_card 43970699, qerror 0.4104534245217416, latency 0.01978898048400879
91976.0
303187.00000000006
171470.0
28977.0
estimating query 2357: predicted 3168300.159480302, true_card 518370, qerror 6.112043828694373, latency 0.009637594223022461
91976.0
328064.0
171470.0
28977.0
79851.0
estimating query 2358: predicted 17607636.311765254, true_card 46537849, qerror 0.3783508840678316, latency 0.011657476425170898
171470.0
28977.0
91976

171470.0
28977.0
79851.0
91976.0
328064.0
303187.00000000006
11102.0
estimating query 2396: predicted 102526836.69299683, true_card 319981016, qerror 0.3204153733076366, latency 0.013665199279785156
171470.0
28977.0
91976.0
328064.0
303187.00000000006
11102.0
estimating query 2397: predicted 80741344.16186658, true_card 3923087, qerror 20.581074078109044, latency 0.012082815170288086
171470.0
28977.0
79851.0
91976.0
303187.00000000006
11102.0
estimating query 2398: predicted 5194792.357872574, true_card 12304064, qerror 0.4222013440333676, latency 0.01219320297241211
171470.0
28977.0
79851.0
91976.0
328064.0
11102.0
estimating query 2399: predicted 5772213.6010164255, true_card 44869387, qerror 0.12864480633569667, latency 0.010260581970214844
171470.0
28977.0
79851.0
91976.0
328064.0
303187.00000000006
estimating query 2400: predicted 35656799.09533178, true_card 226229172, qerror 0.1576136215329992, latency 0.011255264282226562
171470.0
28977.0
79851.0
91976.0
328064.0
303187.0000000

303187.00000000006
171470.0
28977.0
11102.0
estimating query 2452: predicted 5154367.588215498, true_card 200752, qerror 25.675298817523604, latency 0.010971784591674805
171470.0
28977.0
79851.0
91976.0
328064.0
11102.0
estimating query 2453: predicted 6626.256, true_card 67369732, qerror 9.835657354255173e-05, latency 0.009767770767211914
171470.0
28977.0
91976.0
11102.0
328064.0
estimating query 2454: predicted 5157.881793032939, true_card 577029, qerror 0.008938687298269132, latency 0.009581565856933594
171470.0
28977.0
91976.0
11102.0
79851.0
estimating query 2455: predicted 251853.64716472683, true_card 3246110, qerror 0.07758629472344647, latency 0.009219646453857422
171470.0
28977.0
79851.0
91976.0
328064.0
303187.00000000006
estimating query 2456: predicted 40132.423, true_card 289290682, qerror 0.00013872698118911416, latency 0.010889530181884766
91976.0
303187.00000000006
171470.0
28977.0
328064.0
estimating query 2457: predicted 17341.817996646532, true_card 3006637, qerror 

303187.00000000006
28977.0
estimating query 2499: predicted 4914.567155497531, true_card 185940, qerror 0.026430930168320592, latency 0.010309219360351562
28977.0
79851.0
90373.0
328064.0
estimating query 2500: predicted 79406.87542402367, true_card 3408749, qerror 0.023295019792898705, latency 0.010544061660766602
28977.0
90373.0
328064.0
estimating query 2501: predicted 4748.293857156175, true_card 110685, qerror 0.042899163004527945, latency 0.007365226745605469
28977.0
79851.0
90373.0
estimating query 2502: predicted 75375.31437320611, true_card 210347, qerror 0.3583379576281388, latency 0.004639148712158203
91976.0
303187.00000000006
174305.0
328064.0
estimating query 2503: predicted 316988.6758610943, true_card 3005809, qerror 0.10545868877932507, latency 0.003954887390136719
90373.0
303187.00000000006
28977.0
79851.0
174305.0
estimating query 2504: predicted 59203.833247681876, true_card 6254690, qerror 0.009465510400624471, latency 0.010459184646606445
90373.0
303187.0000000000

estimating query 2557: predicted 12713.632711892807, true_card 249157, qerror 0.051026592517540374, latency 0.01073455810546875
90373.0
303187.00000000006
174305.0
28977.0
328064.0
estimating query 2558: predicted 18729.36594286876, true_card 659522, qerror 0.028398394508248034, latency 0.010763168334960938
90373.0
11102.0
303187.00000000006
28977.0
328064.0
estimating query 2559: predicted 1357.983073332733, true_card 87188, qerror 0.015575343778188892, latency 0.011362791061401367
90373.0
11102.0
328064.0
303187.00000000006
28977.0
174305.0
estimating query 2560: predicted 36375.81762654243, true_card 436286, qerror 0.08337608272221074, latency 0.011836528778076172
91976.0
174305.0
estimating query 2561: predicted 58561.472625585404, true_card 169983, qerror 0.34451370210894855, latency 0.0015704631805419922
91976.0
11102.0
174305.0
estimating query 2562: predicted 320721.16133767593, true_card 33456, qerror 9.586357046200261, latency 0.0022284984588623047
91976.0
303187.00000000006


In [29]:
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(qerror, i)}")
print(f"total inference time: {np.sum(latency)}")

q-error 50% percentile is 0.699040521310876
q-error 90% percentile is 21.028146264792486
q-error 95% percentile is 55.98883464625576
q-error 99% percentile is 5211.324412423705
q-error 100% percentile is 82144376.20976338
total inference time: 16.16164231300354


In [34]:
with open("stats_CEB_no_condition.txt", "w") as f:
    for p in pred:
        f.write(str(p)+"\n")